In [12]:
import syllables 
import pandas as pd
import random

In [8]:
words_df = pd.read_csv('words.csv')

In [9]:
type(words_df)

pandas.core.frame.DataFrame

In [10]:
print(words_df)

           word  freq             pos  syll
0        people   372            noun     2
1       history   187            noun     3
2           way   185            noun     1
3           art   183            noun     1
4         world   169            noun     1
...         ...   ...             ...   ...
1520  temporary     1  adjective,noun     4
1521   tomorrow     1     adverb,noun     3
1522       wake     1       verb,noun     1
1523       wrap     1       verb,noun     1
1524  yesterday     1     adverb,noun     3

[1525 rows x 4 columns]


In [13]:
# generate random 16 numbers
random_wordchoice = [random.randint(0, len(words_df)) for p in range(0, 16)]
print(random_wordchoice)

[12, 277, 118, 436, 707, 1404, 685, 403, 661, 1006, 285, 434, 206, 776, 676, 1045]


In [14]:
words_array = [0 for _ in range(16)]
i = 0
for word_index in random_wordchoice:
  words_array[i] = words_df["word"][word_index]
  i +=1

In [17]:
def create_poems(arr):
  j = 0
  poem = [0 for _ in range(4)]
  for i in range(1,5):
    temp =  arr[4*(i-1):4*i]
    poem[i-1] = temp
    # print(poem[i-1])
    verse=' '.join(poem[i-1])
    print(verse)

In [18]:
create_poems(words_array)

computer candidate soup ambition
shot combine table courage
view pound employer wedding
resource plenty tool row
